In [1]:
import cv2 as cv
import numpy as np
import os
from PIL import ImageFont, ImageDraw, Image
from datetime import datetime

In [2]:
def select_camera():
    """
    사용자가 관측 지점을 선택하고 해당 카메라의 VideoCapture 객체를 반환하는 함수
    Returns:
        tuple: 선택된 관측 지점 이름과 해당 카메라의 VideoCapture 객체
    """
    # 카메라 정보 리스트
    cameras = [
        {"name": "불당현대 사거리", "url": "rtsp://210.99.70.120:1935/live/cctv048.stream"},
        {"name": "시청 사거리", "url": "rtsp://210.99.70.120:1935/live/cctv045.stream"},
        {"name": "예술의 전당", "url": "rtsp://210.99.70.120:1935/live/cctv036.stream"}
    ]

    # 사용자에게 보여줄 카메라 선택 목록 생성 및 출력
    prompt = "관측 지점 선택: " + ", ".join([f"[{i+1}: {cam['name']}]" for i, cam in enumerate(cameras)])
    print(prompt)

    point = 0
    try:
        point = int(input("관측 지점 번호 입력: "))
    except ValueError:
        pass # 유효하지 않은 입력은 아래에서 기본값으로 처리

    # 사용자가 입력한 번호가 유효한 범위 내에 있는지 검증
    if not (1 <= point <= len(cameras)):
        print(f"잘못된 입력입니다. [1: {cameras[0]['name']}] 지점을 관측 지점으로 설정합니다.")
        point = 1

    selected_camera = cameras[point - 1]
    point_name = selected_camera["name"]
    rtsp_url = selected_camera["url"]

    print(f"{point}번 지점({point_name}) 선택됨.")
    # 선택된 카메라 이름과 VideoCapture 객체를 튜플로 반환
    return point_name, cv.VideoCapture(rtsp_url)

In [3]:
def apply_filter(frame, filter_type):
    """
    필터 타입에 따라 프레임에 필터를 적용하는 함수
    filter_type:
        1 - 필터 없음
        2 - 그레이 스케일 필터
        3 - 가우시안 블러 필터
        4 - 엣지 검출 필터
        5 - 세피아 필터
        6 - 좌우 반전 (거울 효과)
    """
    if filter_type == 1:
        return frame
    elif filter_type == 2:
        return cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    elif filter_type == 3:
        return cv.GaussianBlur(frame, (7, 7), 0)
    elif filter_type == 4:
        return cv.Canny(frame, 100, 150)
    elif filter_type == 5:
        kernel = np.array([[0.273, 0.535, 0.132],
                            [0.349, 0.686, 0.165],
                            [0.391, 0.769, 0.180]])
        return cv.transform(frame, kernel)
    elif filter_type == 6:
        return cv.flip(frame, 1)
    return frame

In [4]:
def draw_ui_text(frame, point_name, current_time):
    """프레임에 UI 텍스트(관측 지점, 시간)를 그립니다."""
    h, w = frame.shape[:2]
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    
    # Pillow 이미지로 변환하여 텍스트 렌더링
    frame_pil = Image.fromarray(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(frame_pil)
    
    # 텍스트 위치 계산 및 그리기
    draw.text((10, h - 50), point_name, font=font, fill=TEXT_COLOR)
    draw.text((10, h - 25), current_time, font=font, fill=TEXT_COLOR)
    
    # 다시 OpenCV 이미지로 변환하여 반환
    return cv.cvtColor(np.array(frame_pil), cv.COLOR_RGB2BGR)

In [11]:
# -------------------------------------------------
# 설정 값 (Constants)
# -------------------------------------------------
# 영상 처리 관련 설정
TARGET_WIDTH = 720
TARGET_HEIGHT = 480
FPS = 20.0
FOURCC = cv.VideoWriter_fourcc(*'XVID')

# 필터 종류 정의
FILTER_NONE = 1
FILTER_GRAY = 2
FILTER_GAUSSIAN = 3
FILTER_CANNY = 4
FILTER_SEPIA = 5
FILTER_FLIP = 6

# UI 및 폰트 설정
FONT_PATH = "c:/Windows/Fonts/NanumBarunGothic.ttf"
FONT_SIZE = 20
TEXT_COLOR = (0, 255, 0) # Green

# 녹화 파일 저장 경로
SAVE_PATH = './videos/'

# -------------------------------------------------
# 초기화
# -------------------------------------------------
point_name, cap = select_camera()

is_recording = False
current_filter = FILTER_NONE  # 기본 필터: 필터 없음

# 녹화 영상 저장 폴더 생성
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# 출력 파일 이름 설정
output_filename = os.path.join(SAVE_PATH, f"{point_name.replace(' ', '_')}_output_{datetime.now().strftime('%Y%m%d%H%M%S')}.avi")

# 카메라 연결 상태 확인
if not cap.isOpened():
    print("카메라 연결 실패")
    exit()

관측 지점 선택: [1: 불당현대 사거리], [2: 시청 사거리], [3: 예술의 전당]
3번 지점(예술의 전당) 선택됨.


In [12]:
"""
키보드 입력 안내
- ESC: 프로그램 종료
- Space: 녹화 시작/중지
- 숫자 1~6: 필터 변경
"""

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패")
        break
    
    frame = cv.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))
    
    # 1. 필터 적용
    frame_filtered = apply_filter(frame, current_filter)

    # 2. UI 텍스트 그리기
    current_time_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    frame_with_text = draw_ui_text(frame_filtered, point_name, current_time_str)
    
    frame_ui = frame_with_text.copy()
    
    # 3. 녹화 처리
    if is_recording:
        if 'out' not in locals():
            out = cv.VideoWriter(output_filename, FOURCC, FPS, (TARGET_WIDTH, TARGET_HEIGHT))
        out.write(frame_with_text) # 텍스트가 포함된 프레임 녹화
        cv.putText(frame_ui, "REC", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        if 'out' in locals():
            out.release()
            del out
            print(f"녹화된 파일이 저장되었습니다: {output_filename}")

    # 4. 화면 출력
    cv.imshow('Traffic Lens', frame_ui)

    # 5. 키보드 입력 처리
    key = cv.waitKey(1) & 0xFF
    if key == 27: # ESC
        break
    elif key == 32:  # Space
        is_recording = not is_recording
        print(f"녹화 {'시작' if is_recording else '중지'}")
    elif ord('1') <= key <= ord('6'):
        current_filter = int(chr(key))
        print(f"필터 {current_filter}로 변경됨.")

# 종료 시 리소스 해제
if 'out' in locals():
    out.release()
cap.release()
cv.destroyAllWindows()

녹화 시작
필터 2로 변경됨.
필터 3로 변경됨.
필터 4로 변경됨.
필터 5로 변경됨.
필터 6로 변경됨.
필터 1로 변경됨.
녹화 중지
녹화된 파일이 저장되었습니다: ./videos/예술의_전당_output_20250916193825.avi
